# Dataset

In [ ]:
!pip install eurostat

In [ ]:
import pandas as pd
import eurostat
df = eurostat.get_data_df("ei_bsco_m")
df

In [ ]:
df[["indic", "s_adj", "unit", "geo\\time"]].describe()

In [ ]:
df["indic"].unique()

Spalten aufräumen

In [ ]:
df = df.rename(columns={"geo\\time": "country"})

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f.split("M")[0] + "-" + f.split("M")[1] + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

In [ ]:
realnames_i = eurostat.get_dic('indic')
[f'{i}: {realnames_i[i]}' for i in df["indic"].unique()]

# Zeitreihen

In [ ]:
# Länder wählen
cci_mix = df[(df["country"].isin(["DE", "EA19", "SE"])) & 
             (df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
# nur bestimmte Spalte behalten und transponieren (Zeit nach unten)
cci_mix = cci_mix[[c for c in cci_mix.columns 
                     if (isinstance(c, datetime) and c.year>=2000) or c == "country"]].set_index("country").transpose()
# Index in Zeit wandeln
cci_mix.index = pd.DatetimeIndex(cci_mix.index)

In [ ]:
cci_mix.plot()

In [ ]:
cci_mix.resample("Q").mean().plot()

# Balkendiagramm und Histogramm

## Barchart

In [ ]:
de_cci = df[(df["country"] == "DE") & (df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
de_cci = de_cci[[c for c in de_cci.columns if isinstance(c, datetime) and (c.year==2020 or c.year==2019)]]
de_cci.index = ["Consumer confidence indicator"]
de_cci.transpose()[::-1].plot.bar(figsize=(16,9))

In [ ]:
cci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
cci = cci[["country", datetime(2020,10,1)]].set_index("country")
cci

In [ ]:
cci.plot.bar(figsize=(16,9))

In [ ]:
realnames_g = eurostat.get_dic('geo')
realnames_g['DE'] = "Germany"
realnames_g['EA19'] = "EURO"
realnames_g['EU27_2020'] = "EU"

In [ ]:
cci.index = [realnames_g[i] for i in cci.index]
cci.sort_values(datetime(2020, 10, 1)).plot.barh(figsize=(9, 9))

## Histogramm

In [ ]:
cci.plot.hist()

In [ ]:
cci.plot.hist(bins=20)

# Fieberkurven

# Boxplots

## 10 Jahre

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
csmci

In [ ]:
columns = [datetime(y, m, 1) for y in range(2010, 2021) for m in range(1, 13)]
# die letzten beiden Spalten sind für November/Dezember 2020, da gibt es heute noch keine Daten
csmci_unwrapped = pd.melt(csmci[["country"] + columns[:-2]].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")
csmci_unwrapped

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

## 20 Jahre

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "NSA")]
columns = [datetime(y, m, 1) for y in range(2000, 2021) for m in range(1, 13)]
# die letzten beiden Spalten sind für November/Dezember 2020, da gibt es heute noch keine Daten
csmci_unwrapped = pd.melt(csmci[["country"] + columns[:-2]].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")

In [ ]:
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]

In [ ]:
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values

In [ ]:
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

## Violinplot

In [ ]:
plt.figure(figsize=(8, 16))
sns.violinplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

In [ ]:
sns.violinplot(y="country", x="csmci", 
               data=csmci_unwrapped[csmci_unwrapped["country"].isin(["Greece", "Germany", "Sweden", "Denmark"])],
               order=["Greece", "Germany", "Sweden", "Denmark"], palette="viridis")